In [9]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders["order_date"] = pd.to_datetime(orders["order_date"], format="%d-%m-%Y")

print(orders.head())
print(orders.info())


   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   
3         4      319            224 2023-10-04       1155.97   
4         5     1064            293 2023-12-25       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000

In [10]:
users = pd.read_json("users.json")
display(users.head())
display(users.info())

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     3000 non-null   int64 
 1   name        3000 non-null   object
 2   city        3000 non-null   object
 3   membership  3000 non-null   object
dtypes: int64(1), object(3)
memory usage: 93.9+ KB


None

In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("restaurants_data.db")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()
    conn.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

conn.close()

display(restaurants.head())
display(restaurants.info())

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    object 
 2   cuisine          500 non-null    object 
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 15.8+ KB


None

In [15]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [16]:
final_data= pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [17]:
final_data.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)


In [18]:
print(final_data.shape)
print(final_data.isnull().sum())
print(final_data.duplicated().sum())


(10000, 12)
order_id             0
user_id              0
restaurant_id        0
order_date           0
total_amount         0
restaurant_name_x    0
name                 0
city                 0
membership           0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64
0


In [20]:
df = pd.read_csv("final_food_delivery_dataset.csv")

gold = df[df["membership"] == "Gold"]

gold.groupby("city")["total_amount"].mean().sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [22]:
avg_order_value = df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print("\nAverage Order Value by Cuisine:")
display(avg_order_value)


Average Order Value by Cuisine:


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [24]:
user_spending = df.groupby('user_id')['total_amount'].sum()
high_spending_users = (user_spending > 1000).sum()
high_spending_users

np.int64(2544)

In [26]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]
df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)

rating_revenue = df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)
rating_revenue

rating_range
4.6 – 5.0    2197030.75
3.0 – 3.5    2136772.70
4.1 – 4.5    1960326.26
3.6 – 4.0    1717494.41
Name: total_amount, dtype: float64

In [27]:
gold_total_revenue = df[df['membership'] == 'Gold'].groupby('city')['total_amount'].sum().sort_values(ascending=False)
gold_total_revenue

city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [28]:
cusine = df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
cusine

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [29]:
pct_gold = round(
    (df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100
)
pct_gold


50

In [31]:
options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
analysis = df.groupby('restaurant_name_x').agg(avg_order=('total_amount', 'mean'), orders=('order_id', 'count'))
result = analysis[analysis.index.isin(options) & (analysis['orders'] < 20)].sort_values('avg_order', ascending=False)
display(result)

,avg_order,orders
restaurant_name_x,,
Ruchi Foods Chinese,686.603158,19


In [45]:

df['order_date'] = pd.to_datetime(df['order_date'])

df['quarter_label'] = df['order_date'].dt.quarter

quarterly_totals = df.groupby('quarter_label')['total_amount'].sum().reset_index()

quarter_map = {1: 'Q1', 2: 'Q2', 3: 'Q3', 4: 'Q4'}
quarterly_totals['quarter'] = quarterly_totals['quarter_label'].map(quarter_map)
quarterly_totals = quarterly_totals.sort_values(by='total_amount', ascending=False)

print(quarterly_totals[['quarter', 'total_amount']].to_string(index=False))


quarter  total_amount
     Q3    2037385.10
     Q4    2018263.66
     Q1    2010626.64
     Q2    1945348.72


In [48]:
requested_pairs = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

mask = df.apply(lambda row: (row['membership'], row['cuisine']) in requested_pairs, axis=1)
combinations_revenue = df[mask].groupby(['membership', 'cuisine'])['total_amount'].sum().reset_index()

display(combinations_revenue.sort_values(by='total_amount', ascending=False))


,membership,cuisine,total_amount
1,Gold,Italian,1005779.05
3,Regular,Indian,992100.27
0,Gold,Indian,979312.31
2,Regular,Chinese,952790.91


In [33]:
gold_orders_count = df[df['membership'] == 'Gold'].shape[0]
gold_orders_count

4987

In [34]:
hyderabad_revenue = round(df[df['city'] == 'Hyderabad']['total_amount'].sum())
hyderabad_revenue

1889367

In [35]:
unique_users = df['user_id'].nunique()
unique_users

2883

In [37]:
avg_gold_order = df[df['membership'] == 'Gold']['total_amount'].mean()
print(round(avg_gold_order, 2))

797.15


In [38]:
high_rated_orders = df[df['rating'] >= 4.5].shape[0]
high_rated_orders

3374

In [39]:
chennai_gold_orders = df[(df['membership'] == 'Gold') & (df['city'] == 'Chennai')].shape[0]
chennai_gold_orders

1337